In [1]:
%cd ../../

/home/PJWSTK/s14028/engineer/Practical


In [18]:
import os

import pandas as pd
import keras
import scipy.io as mat

from common import *
from augmentation import add_pmap
from testing import test_model
from testing import best_cop_diff
from testing import mse
from testing import mae

from CNN.CNN_v6 import CNN_v6

In [3]:
perspective = mat.loadmat("mall_dataset/perspective_roi.mat")["pMapN"]

perspective /= np.min(perspective)
perspective = np.round(perspective).astype(np.uint8)

train, test = data_sets()
image_tensors = train[0], test[0]
person_coo_tensors = train[1], test[1]
count_matrix = train[2], test[2]

image_train, image_test = image_tensors
person_coo_train, person_coo_test = person_coo_tensors
count_train, count_test = count_matrix
count_train = count_train.astype(np.uint16)
count_test = count_test.astype(np.uint16)

image_train = add_pmap(image_train, perspective)
image_test = add_pmap(image_test, perspective)

In [4]:
cnn = CNN_v6((480, 640, 4), split_into_parts=20)

images = cnn._prepare_images(image_test)
anwsers = cnn._prepare_anwsers(person_coo_test)

In [5]:
cop = np.linspace(0, 1, 11)[1:-1]

In [6]:
prefix = "CNN/CNN_v6/weights/"

def get_weights_paths(prefix):
    for directory, subdirectories, files in os.walk(prefix):
        yield from (os.path.join(directory, file) for file in files)

model_names = get_weights_paths(prefix)
model_names = list(model_names)

In [7]:
models = [CNN_v6((480, 640, 4), split_into_parts=20) for i in range(len(model_names))]

In [8]:
results = {}

for model_name, model in zip(model_names, models):
  model.def_model()
  model.model.fit(images[:1], anwsers[:1])
  
  model.model = keras.utils.multi_gpu_model(model.model, gpus=2, cpu_merge=False)
  model.model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
  
  model.model.load_weights(model_name)
  
  results[model_name] = test_model(model=model, images=image_test, counts=count_test, cut_of_points=cop)

Epoch 1/1
1/1 [==============================] - 2s 2s/step - loss: 0.8961 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 1s/step - loss: 0.8639 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 859ms/step - loss: 0.9138 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 853ms/step - loss: 0.9326 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 1s/step - loss: 0.9078 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 849ms/step - loss: 0.8886 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 1s/step - loss: 0.9281 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 1s/step - loss: 0.9036 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 851ms/step - loss: 0.8626 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 1s/step - loss: 0.8671 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] -

In [31]:
def as_rows(results):
    for name, cut_off_point_to_accuracy_with_difference in results.items():
        yield from ((f"{name}_{cut_off_point}", accuracy, counts - count_test) for cut_off_point, (accuracy, counts) in cut_off_point_to_accuracy_with_difference.items())

rows = as_rows(results)
rows = list(rows)

In [32]:
def calculate_mae_and_mse(rows):
    for *attributes, difference in rows:
        difference_mae = mae(difference)
        difference_mse = mse(difference)
        
        yield (*attributes, difference_mae, difference_mse)

mae_and_mse = calculate_mae_and_mse(rows)
mae_and_mse = list(mae_and_mse)

In [33]:
frame = pd.DataFrame(mae_and_mse, columns=["model", "accuracy", "mae", "mse"])
frame

,model,accuracy,mae,mse
0,CNN/CNN_v6/weights/cnn_v6_1e_6_0_0.1,0.000000,274.373333,75488.640000
1,CNN/CNN_v6/weights/cnn_v6_1e_6_0_0.2,0.000000,162.770000,26715.516667
2,CNN/CNN_v6/weights/cnn_v6_1e_6_0_0.30000000000...,0.000000,90.633333,8356.240000
3,CNN/CNN_v6/weights/cnn_v6_1e_6_0_0.4,2.370590,41.763333,1824.843333
4,CNN/CNN_v6/weights/cnn_v6_1e_6_0_0.5,75.591116,7.030000,74.410000
...,...,...,...,...
130,CNN/CNN_v6/weights/cnn_v6_1e_6_14_0.5,19.588647,25.770000,729.243333
131,CNN/CNN_v6/weights/cnn_v6_1e_6_14_0.6000000000...,47.832949,15.970000,294.950000
132,CNN/CNN_v6/weights/cnn_v6_1e_6_14_0.7000000000...,87.340673,3.900000,24.186667
133,CNN/CNN_v6/weights/cnn_v6_1e_6_14_0.8,68.063846,10.006667,121.806667


In [34]:
frame.loc[frame.mae == frame.mae.min()]

,model,accuracy,mae,mse
114,CNN/CNN_v6/weights/cnn_v6_1e_6_12_0.7000000000...,87.907471,3.69,21.663333


In [35]:
frame.loc[frame.mse == frame.mse.min()]

,model,accuracy,mae,mse
114,CNN/CNN_v6/weights/cnn_v6_1e_6_12_0.7000000000...,87.907471,3.69,21.663333


In [36]:
frame.loc[frame.accuracy == frame.accuracy.max()]

,model,accuracy,mae,mse
114,CNN/CNN_v6/weights/cnn_v6_1e_6_12_0.7000000000...,87.907471,3.69,21.663333
